# Bundle pose estimation by reflected rays
Find rigid pose (R, t) of the mirror bundle and assign observed reflected rays to mirrors.


Algorithm outline:
1. Load mirrors (local centers/normals) and observed rays (two points per ray).
2. Estimate ray normals: `n_est = normalize(d_in - d_out)` from observed directions.
3. Iterate: for current R, t build cost (angle + distance of mirror center to ray), find best mirror?ray assignment (rays < mirrors).
4. Update R with Kabsch on normals; update t to place centers onto closest points of assigned rays.
5. Stop when assignment/cost stabilizes.


In [ ]:
import json
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple, Dict
import itertools as it
import numpy as np


In [ ]:
BUNDLE_CONFIG = Path('bundle_config.txt')
RAYS_CONFIG = Path('rays_config.txt')

# Incoming ray is fixed: along -Z, hits the center of each mirror
D_IN = np.array([0.0, 0.0, -1.0])


In [ ]:
@dataclass
class Mirror:
    name: str
    cut_deg: float
    tilt_deg: float
    height: float
    x: float
    y: float

@dataclass
class BundleConfig:
    fiber_diameter: float = 1.0
    pitch: float = 1.4
    label_offset: float = 0.6
    mirrors: List[Mirror] = None

def _parse_float(value: str, field: str, raw: str) -> float:
    try:
        return float(value)
    except ValueError as exc:
        raise ValueError(f'Failed to read {field} in line: {raw}') from exc

def load_bundle_config(path: Path) -> BundleConfig:
    mirrors: List[Mirror] = []
    globals_cfg = {}
    for raw in path.read_text(encoding='utf-8').splitlines():
        clean = raw.split('#', 1)[0].strip()
        if not clean:
            continue
        if clean.startswith('mirror'):
            parts = [p.strip() for p in clean.split(',') if p.strip()]
            params = {}
            for part in parts:
                if '=' in part:
                    k, v = part.split('=', 1)
                    params[k.strip()] = v.strip()
            name = params.get('mirror') or params.get('name')
            cut = _parse_float(params.get('cut', ''), 'cut', raw)
            tilt = _parse_float(params.get('tilt', '0'), 'tilt', raw)
            height = _parse_float(params.get('height', '0'), 'height', raw)
            x = _parse_float(params.get('x', '0'), 'x', raw)
            y = _parse_float(params.get('y', '0'), 'y', raw)
            mirrors.append(Mirror(name=name, cut_deg=cut, tilt_deg=tilt, height=height, x=x, y=y))
        else:
            if '=' not in clean:
                continue
            k, v = clean.split('=', 1)
            globals_cfg[k.strip()] = float(v.strip())
    return BundleConfig(
        fiber_diameter=float(globals_cfg.get('fiber_diameter', 1.0)),
        pitch=float(globals_cfg.get('pitch', globals_cfg.get('pitch_mm', 1.4))),
        label_offset=float(globals_cfg.get('label_offset', 0.6)),
        mirrors=mirrors,
    )

def load_rays_config(path: Path) -> List[Tuple[np.ndarray, np.ndarray]]:
    rays = []
    for raw in path.read_text(encoding='utf-8').splitlines():
        clean = raw.split('#', 1)[0].strip()
        if not clean:
            continue
        parts = [p.strip() for p in clean.split(',') if p.strip()]
        p0 = p1 = None
        for p in parts:
            if p.startswith('p0='):
                vals = p.split('=', 1)[1].strip().split()
                p0 = np.array([float(v) for v in vals], dtype=float)
            if p.startswith('p1='):
                vals = p.split('=', 1)[1].strip().split()
                p1 = np.array([float(v) for v in vals], dtype=float)
        if p0 is None or p1 is None:
            raise ValueError(f'Missing p0/p1 in line: {raw}')
        rays.append((p0, p1))
    return rays

def mirror_geometry(m: Mirror):
    cut_rad = np.deg2rad(m.cut_deg)
    tilt_rad = np.deg2rad(m.tilt_deg)
    n_local = np.array([
        np.sin(cut_rad) * np.cos(tilt_rad),
        np.sin(cut_rad) * np.sin(tilt_rad),
        np.cos(cut_rad),
    ], dtype=float)
    c_local = np.array([m.x, m.y, m.height], dtype=float)
    return n_local / np.linalg.norm(n_local), c_local


In [ ]:
def kabsch(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    # A, B: 3 x N
    H = A @ B.T
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T
    return R

def distance_point_to_line(p: np.ndarray, a: np.ndarray, d: np.ndarray) -> float:
    # line: a + s d, d must be unit
    diff = p - a
    proj = np.dot(diff, d) * d
    return float(np.linalg.norm(diff - proj))

def best_assignment(cost: np.ndarray):
    m, k = cost.shape
    rays_idx = list(range(k))
    best_total = np.inf
    best_match = None
    for mirrors_subset in it.combinations(range(m), k):
        for perm in it.permutations(rays_idx):
            total = 0.0
            for mi, rj in zip(mirrors_subset, perm):
                total += cost[mi, rj]
                if total >= best_total:
                    break
            else:
                if total < best_total:
                    best_total = total
                    best_match = list(zip(mirrors_subset, perm))
    return best_total, best_match

def solve_pose(bundle: BundleConfig, rays: List[Tuple[np.ndarray, np.ndarray]], *, w_ang=1.0, w_pos=1.0, iters=20):
    normals_local = []
    centers_local = []
    for m in bundle.mirrors:
        n_loc, c_loc = mirror_geometry(m)
        normals_local.append(n_loc)
        centers_local.append(c_loc)
    normals_local = np.stack(normals_local)
    centers_local = np.stack(centers_local)

    d_obs = []
    n_est = []
    ray_anchors = []
    for p0, p1 in rays:
        d = p1 - p0
        d = d / np.linalg.norm(d)
        d_obs.append(d)
        n_est.append((D_IN - d) / np.linalg.norm(D_IN - d))
        ray_anchors.append(p0)
    d_obs = np.stack(d_obs)
    n_est = np.stack(n_est)
    ray_anchors = np.stack(ray_anchors)

    R = np.eye(3)
    t = np.zeros(3)
    best = None

    for _ in range(iters):
        normals_world = (R @ normals_local.T).T
        centers_world = (R @ centers_local.T).T + t
        d_out = d_obs * 0
        for i, n in enumerate(normals_world):
            d_out[i % len(d_obs)] = D_IN - 2 * np.dot(n, D_IN) * n  # placeholder size, we'll use per mirror below

        m = len(bundle.mirrors)
        k = len(rays)
        cost = np.zeros((m, k))
        for mi in range(m):
            n = normals_world[mi]
            d_pred = D_IN - 2 * np.dot(n, D_IN) * n
            d_pred = d_pred / np.linalg.norm(d_pred)
            for rj in range(k):
                ang = 1.0 - np.dot(d_pred, d_obs[rj])
                dist = distance_point_to_line(centers_world[mi], ray_anchors[rj], d_obs[rj])
                cost[mi, rj] = w_ang * ang + w_pos * dist

        total, match = best_assignment(cost)
        if match is None:
            raise RuntimeError('Assignment not found')

        # Update rotation R by aligning normals (Kabsch)
        A = []
        B = []
        closest_pts = []
        for mi, rj in match:
            A.append(normals_local[mi])
            B.append(n_est[rj])
            d = d_obs[rj]
            a = ray_anchors[rj]
            c_loc = centers_local[mi]
            c_world = R @ c_loc + t
            s = np.dot(d, c_world - a)
            closest_pts.append(a + s * d)
        A = np.stack(A).T
        B = np.stack(B).T
        R_new = kabsch(A, B)
        closest_pts = np.stack(closest_pts)
        centers_assigned = centers_local[[mi for mi, _ in match]]
        t_new = closest_pts.mean(axis=0) - (R_new @ centers_assigned.T).T.mean(axis=0)

        dR = np.linalg.norm(R - R_new)
        dt = np.linalg.norm(t - t_new)
        R, t = R_new, t_new
        best = (total, match, R, t)
        if dR < 1e-6 and dt < 1e-6:
            break

    return best


In [ ]:
bundle = load_bundle_config(BUNDLE_CONFIG)
rays = load_rays_config(RAYS_CONFIG)
total, match, R, t = solve_pose(bundle, rays, w_ang=1.0, w_pos=1.0, iters=20)
print(f'Total cost: {total:.4f}')
print('Rotation R:')
print(R)
print('Translation t:')
print(t)
print('Assignment (mirror idx -> ray idx):', match)


In [ ]:
import plotly.graph_objects as go

AXIS_LOCAL = np.array([0.0, 0.0, 1.0])
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


def compute_world(bundle, R, t):
    normals = []
    centers = []
    for m in bundle.mirrors:
        n_loc, c_loc = mirror_geometry(m)
        normals.append(n_loc)
        centers.append(c_loc)
    normals = np.stack(normals)
    centers = np.stack(centers)
    normals_w = (R @ normals.T).T
    centers_w = (R @ centers.T).T + t
    return normals_w, centers_w


def ellipse_boundary(center, normal, axis_world, fiber_diameter, samples=64):
    minor_r = fiber_diameter * 0.5
    cos_theta = float(np.clip(np.dot(normal, axis_world), -1.0, 1.0))
    major_r = minor_r / max(abs(cos_theta), 1e-6)
    tangent = axis_world - cos_theta * normal
    if np.linalg.norm(tangent) < 1e-8:
        tangent = np.cross(normal, np.array([1.0, 0.0, 0.0]))
    if np.linalg.norm(tangent) < 1e-8:
        tangent = np.cross(normal, np.array([0.0, 1.0, 0.0]))
    e1 = tangent / np.linalg.norm(tangent)
    e2 = np.cross(normal, e1)
    theta = np.linspace(0.0, 2 * np.pi, samples)
    boundary = center + major_r * np.cos(theta)[:, None] * e1 + minor_r * np.sin(theta)[:, None] * e2
    return boundary


def plot_solution(bundle, rays, R, t, match, ray_len=30.0):
    normals_w, centers_w = compute_world(bundle, R, t)
    axis_world = R @ AXIS_LOCAL
    fig = go.Figure()

    # Observed rays
    for j, (p0, p1) in enumerate(rays):
        fig.add_trace(go.Scatter3d(
            x=[p0[0], p1[0]], y=[p0[1], p1[1]], z=[p0[2], p1[2]],
            mode='lines', line=dict(color='rgba(0,0,150,0.6)', width=6),
            name=f'obs ray {j}'
        ))

    # Predicted rays and mirror apertures
    for i, (c, n) in enumerate(zip(centers_w, normals_w)):
        color = COLORS[i % len(COLORS)]
        d_pred = D_IN - 2 * np.dot(n, D_IN) * n
        d_pred = d_pred / np.linalg.norm(d_pred)
        end = c + d_pred * ray_len
        fig.add_trace(go.Scatter3d(
            x=[c[0], end[0]], y=[c[1], end[1]], z=[c[2], end[2]],
            mode='lines', line=dict(color=color, width=4),
            name=f'mirror {i} pred'
        ))
        boundary = ellipse_boundary(c, n, axis_world, bundle.fiber_diameter)
        fig.add_trace(go.Scatter3d(
            x=boundary[:, 0], y=boundary[:, 1], z=boundary[:, 2],
            mode='lines', line=dict(color=color, width=2),
            name=f'mirror {i} aperture', showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=[c[0]], y=[c[1]], z=[c[2]], mode='markers+text',
            marker=dict(color=color, size=4), text=[f'M{i}'], textposition='top center',
            showlegend=False
        ))

    fig.update_layout(
        height=900, width=900,
        title='Observed rays vs predicted reflected rays',
        scene=dict(
            aspectmode='cube',
            aspectratio=dict(x=1, y=1, z=1),
            xaxis_title='X', yaxis_title='Y', zaxis_title='Z'
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )
    fig.show()

plot_solution(bundle, rays, R, t, match)
